In [7]:
import numpy as np
import math 

L, d_k, d_v = 4,8,8
q = np.random.rand(L, d_k)
k = np.random.rand(L, d_k)
v = np.random.rand(L, d_v)

In [8]:
print("Query (q):", q)
print("Key (k):", k)
print("Value (v):", v)

Query (q): [[0.95615214 0.2516468  0.25471824 0.67409922 0.16604337 0.52229191
  0.00202645 0.29708519]
 [0.0257909  0.70716937 0.38376479 0.93183365 0.09950033 0.50568616
  0.39259569 0.33317069]
 [0.9649239  0.22719879 0.77705208 0.71782392 0.93203625 0.50669318
  0.84679911 0.10830455]
 [0.32609188 0.59047332 0.87358978 0.24973147 0.34850938 0.19769565
  0.81442511 0.4130676 ]]
Key (k): [[0.16426018 0.79176323 0.1876594  0.01290285 0.15181112 0.69171364
  0.91690193 0.29677066]
 [0.07050061 0.41016288 0.7878722  0.64135083 0.63667069 0.77341149
  0.92101247 0.53107275]
 [0.90576254 0.54968273 0.77062321 0.12486301 0.18896326 0.60341883
  0.40832629 0.43391489]
 [0.08566413 0.50429736 0.32512693 0.21547856 0.70325699 0.89528526
  0.13130027 0.37664254]]
Value (v): [[0.78203122 0.47245608 0.89768033 0.47811463 0.30296051 0.01657495
  0.10470121 0.37804387]
 [0.64491027 0.95649913 0.88476863 0.05037058 0.525048   0.3222886
  0.93384124 0.46115092]
 [0.59487447 0.66990116 0.97107586 0.5

Self Attention

                        self attention = softmax((Q.K^t / root d_k) + M) V

In [9]:
np.matmul(q, k.T)

array([[0.88930837, 1.47294674, 1.76110854, 1.13341485],
       [1.47192981, 2.18483864, 1.45298617, 1.38413729],
       [1.79402272, 3.05652356, 2.56195731, 1.86562432],
       [1.74722923, 2.45787293, 2.0212617 , 1.34814658]])

In [10]:
# If we didn't do scaling
q.var(),k.var(),np.matmul(q, k.T).var()

(0.08606963863565467, 0.0808268726833457, 0.29883294430311363)

In [11]:
# With scaling
scaled = np.matmul(q, k.T) / math.sqrt(d_k)      # Dividing by root d_k for scaling
q.var(),k.var(),scaled.var()

(0.08606963863565467, 0.0808268726833457, 0.03735411803788921)

In [12]:
scaled

array([[0.31441799, 0.52076531, 0.6226459 , 0.40072266],
       [0.52040578, 0.77245711, 0.51370819, 0.48936643],
       [0.63428282, 1.08064427, 0.90578869, 0.6595978 ],
       [0.61773882, 0.86898931, 0.71462393, 0.47664179]])

Masking -
1) This is to ensure words don't get context from words generated in the future
2) Not required in encoders but required in decoders

In [1]:
mask = np.tril(np.ones((L, L)))  # Lower triangular matrix
mask

NameError: name 'np' is not defined